In [ ]:
"""
fetch_prices.py

Fetches daily stock price data (non-adjusted) from Alpha Vantage and stores it in PostgreSQL.
"""

import os
import requests
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv
from models.price_data_model import Base, PriceData

# Load environment variables
load_dotenv()

In [15]:


# Configuration
API_KEY = os.getenv("ALPHA_VANTAGE_API_KEY")
BASE_URL = 'https://www.alphavantage.co/query'
DATABASE_URL = os.getenv("DATABASE_URL")

# SQLAlchemy setup
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

def fetch_daily_prices(ticker):
    """Fetch daily stock prices (not adjusted) from Alpha Vantage and store in DB."""
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': ticker,
        'outputsize': 'full',
        'apikey': API_KEY
    }

    response = requests.get(BASE_URL, params=params)
    if response.status_code != 200:
        raise Exception(f"API request failed: {response.status_code}")

    json_data = response.json()

    # Debug: Check if API returned valid data
    if 'Time Series (Daily)' not in json_data:
        print("API Error Response:", json_data)
        raise ValueError(f"No data returned for ticker {ticker}")

    data = json_data['Time Series (Daily)']
    records = []

    for date_str, values in data.items():
        record = PriceData(
            ticker=ticker,
            date=datetime.strptime(date_str, '%Y-%m-%d').date(),
            open=float(values['1. open']),
            high=float(values['2. high']),
            low=float(values['3. low']),
            close=float(values['4. close']),
            adjusted_close=None,  # Not available in this endpoint
            volume=int(values['5. volume']),
            source_api='Alpha Vantage',
            interval='1d',
            currency='USD',
            data_type='historical',
            created_at=datetime.utcnow()
        )
        records.append(record)

    session.bulk_save_objects(records)
    session.commit()
    print(f"Inserted {len(records)} price records for {ticker}")

if __name__ == '__main__':
    Base.metadata.create_all(engine)
    fetch_daily_prices('NVDA')  # Use demo-compatible ticker


Inserted 6441 price records for NVDA
